In [1]:
from bs4 import BeautifulSoup
import requests

def text_by_link(url):
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    my_text = [text.get_text().replace('\n', ' ') for text in soup.findAll('p') ]
    return my_text


lotr_books = {}

lotr_books['0'] = text_by_link("https://ae-lib.org.ua/texts-c/tolkien__the_hobbit__en.htm")[29:-2]

lotr_1 = text_by_link("https://ae-lib.org.ua/texts-c/tolkien__the_lord_of_the_rings_1__en.htm")
# SKIPED PROLOGUE
prologue = lotr_1[37:106]
lotr_books['1'] = lotr_1[109:2054]
lotr_books['2'] = lotr_1[2057:-6]

lotr_2 = text_by_link("https://ae-lib.org.ua/texts-c/tolkien__the_lord_of_the_rings_2__en.htm")
lotr_books['3'] = lotr_2[28:1760]
lotr_books['4'] = lotr_2[1763:-5]

lotr_3 = text_by_link("https://ae-lib.org.ua/texts-c/tolkien__the_lord_of_the_rings_3__en.htm")
lotr_books['5'] = lotr_3[27:1251]
lotr_books['6'] = lotr_3[1254:-2]

# DIDN'T FIND TEXT OF APPENDICES

lotr_books['7'] = text_by_link("https://www.ae-lib.org.ua/texts-c/tolkien__the_adventures_of_tom_bombadil__en.htm")[23:-2]

lotr_books['8'] = text_by_link("https://www.ae-lib.org.ua/texts-c/tolkien__unfinished_tales__en.htm")[1716:2228]



In [2]:
import re

def data_by_paragraphs(list_of_paragraphs):

    word_length_counter = {}
    word_length_counter_without_articles = {}
#     long_words = []
    final_data = {}

    for k in range(1, 36+1):
        word_length_counter[str(k)] = 0
        word_length_counter_without_articles[str(k)] = 0

    for paragraph in list_of_paragraphs:
        words = re.sub("[^\w]", " ",  paragraph).split()
        for w in words:
            if w not in ['a', 'an', 'the']:
                word_length_counter_without_articles[str(len(w))] += 1
            word_length_counter[str(len(w))] += 1
#             if len(w) >= 13:
#                 long_words.append(str.lower(w))
                
#     long_words = list(set(long_words))

    symbols_counter, words_counter, symbols_without_articles_counter, words_without_articles_counter = 0, 0, 0, 0

    for key, value in word_length_counter.items():
        symbols_counter += int(key) * value 
        words_counter += value
        
    for key, value in word_length_counter_without_articles.items():
        symbols_without_articles_counter += int(key) * value
        words_without_articles_counter += value

    final_data['full_symbols_cnt'] = sum([len(s) for s in list_of_paragraphs])
    
    final_data['symbols_cnt'] = symbols_counter
    final_data['words_cnt'] = words_counter
    final_data['average_word_lenght'] = round(symbols_counter/words_counter, 3)

    final_data['symbols_without_articles_cnt'] = symbols_without_articles_counter
    final_data['words_without_articles_cnt'] = words_without_articles_counter
    final_data['average_word_without_articles_lenght'] = round(symbols_without_articles_counter/words_without_articles_counter, 3)
#     final_data['top_long_words'] = long_words
    
    return final_data


In [ ]:
'''
DATASET #1: STRUCTURE AND VOLUMES
'''

In [3]:
import pandas as pd

df_list = []

for key, value in lotr_books.items():
    my_text = value
    my_dict = data_by_paragraphs(my_text)
    for k, v in my_dict.items():
        my_dict[k] = [v]
            
    my_df = pd.DataFrame.from_dict(my_dict)
    number = int(key)
    my_df2 = my_df.assign(number=number)
    df_list.append(my_df2)

df_lotr = pd.concat(df_list).reset_index()

df_lotr.loc[df_lotr['number'] == 0, ['book_name']] = 'The Hobbit, or There and Back Again'
df_lotr.loc[df_lotr['number'] == 1, ['book_name']] = 'The Fellowship of the Ring'
df_lotr.loc[df_lotr['number'] == 2, ['book_name']] = 'The Fellowship of the Ring'
df_lotr.loc[df_lotr['number'] == 3, ['book_name']] = 'The Two Towers'
df_lotr.loc[df_lotr['number'] == 4, ['book_name']] = 'The Two Towers'
df_lotr.loc[df_lotr['number'] == 5, ['book_name']] = 'The Return of the King'
df_lotr.loc[df_lotr['number'] == 6, ['book_name']] = 'The Return of the King'
df_lotr.loc[df_lotr['number'] == 7, ['book_name']] = 'The Adventures of Tom Bombadil'
df_lotr.loc[df_lotr['number'] == 8, ['book_name']] = 'The Third Age'

df_lotr.loc[df_lotr['number'] == 0, ['volume_name']] = 'The Hobbit, or There and Back Again'
df_lotr.loc[df_lotr['number'] == 1, ['volume_name']] = 'The Lord of the Rings, book I'
df_lotr.loc[df_lotr['number'] == 2, ['volume_name']] = 'The Lord of the Rings, book II'
df_lotr.loc[df_lotr['number'] == 3, ['volume_name']] = 'The Lord of the Rings, book III'
df_lotr.loc[df_lotr['number'] == 4, ['volume_name']] = 'The Lord of the Rings, book IV'
df_lotr.loc[df_lotr['number'] == 5, ['volume_name']] = 'The Lord of the Rings, book V'
df_lotr.loc[df_lotr['number'] == 6, ['volume_name']] = 'The Lord of the Rings, book VI'
df_lotr.loc[df_lotr['number'] == 7, ['volume_name']] = 'The Adventures of Tom Bombadil'
df_lotr.loc[df_lotr['number'] == 8, ['volume_name']] = 'Unfinished Tales of Numenor and Middle-Earth, part III'

df_lotr


,index,full_symbols_cnt,symbols_cnt,words_cnt,average_word_lenght,symbols_without_articles_cnt,words_without_articles_cnt,average_word_without_articles_lenght,number,book_name,volume_name
0,0,503617,392856,96690,4.063,373785,89035,4.198,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
1,0,472666,366314,90478,4.049,349980,83763,4.178,1,The Fellowship of the Ring,"The Lord of the Rings, book I"
2,0,461710,357260,88922,4.018,339789,82011,4.143,2,The Fellowship of the Ring,"The Lord of the Rings, book II"
3,0,464547,358967,88835,4.041,342301,82187,4.165,3,The Two Towers,"The Lord of the Rings, book III"
4,0,349155,268690,67355,3.989,257747,62828,4.102,4,The Two Towers,"The Lord of the Rings, book IV"
5,0,369137,286329,71124,4.026,271649,65466,4.149,5,The Return of the King,"The Lord of the Rings, book V"
6,0,336726,259493,65881,3.939,247453,61015,4.056,6,The Return of the King,"The Lord of the Rings, book VI"
7,0,51068,40398,9935,4.066,38581,9157,4.213,7,The Adventures of Tom Bombadil,The Adventures of Tom Bombadil
8,0,267850,208607,49404,4.222,196306,44856,4.376,8,The Third Age,"Unfinished Tales of Numenor and Middle-Earth, ..."


In [7]:
df_lotr.to_csv('lotr_books.csv')

In [ ]:
'''
DATASET #2: CHARACTERS
'''

In [4]:
from collections import Counter


def words_info(list_of_paragraphs):
#     list of words in original spelling
    words_full = re.sub("[^\w]", " ",  ' '.join(list_of_paragraphs)).split()  
#     list of words in lower spelling
    lower_words_full = [str.lower(w) for w in words_full]
    
#     counter by lower spelling
    frequency_lower_counter = Counter(lower_words_full)
    frequency_lower_dict = dict(frequency_lower_counter)

#     counter of words with capital first letter
    capital_pattern = r"\b[A-ZÉ][a-z][^\W]*\b"
    words_with_capital_full = [w for w in words_full if re.match(capital_pattern, w)]
    frequency_capital_counter = Counter(words_with_capital_full)
    frequency_capital_dict = dict(frequency_capital_counter)
    
    lower_dict = {}
    for key, value in frequency_lower_dict.items():
        lower_dict[key] = {}
        lower_dict[key]['full_counter'] = value
    lower_df = pd.DataFrame.from_dict(lower_dict, orient='index').reset_index().rename(columns={'index':'word'})

    special_dict = {}
    for key2, value2 in frequency_capital_dict.items():
        special_dict[str.lower(key2)] = {}
        special_dict[str.lower(key2)]['special_counter'] = value2
        special_dict[str.lower(key2)]['special_view'] = key2
    special_df = pd.DataFrame.from_dict(special_dict, orient='index').reset_index().rename(columns={'index':'word'})
        
    final_df = pd.merge(lower_df, special_df, on='word', how='outer')   
    
    return final_df


#     to sum counters for main character's name and his pseudonyms

def replace_pseudonym(final_df, pseudo_word, correct_word):
    final_df2 = final_df.copy(deep=True)
    final_df2.loc[final_df2['word'] == pseudo_word, 'word'] = correct_word
    final_df2 = final_df2.groupby('word', as_index=False).agg({'full_counter':'sum', 'special_counter':'sum'})
    final_df = pd.merge(final_df2, final_df[['word','special_view']], on='word', how='outer')   
    final_df.drop(final_df[final_df['word'] == pseudo_word].index, inplace=True)
    return final_df


replace_couples = [
    ['strider','aragorn'],
    ['mithrandir','gandalf'],
    ['peregrin','pippin'],
    ['sméagol','gollum'],
    ['meriadoc','merry'],
    ['elessar','aragorn']
]


In [11]:
import functools as ft

df_list = [] 

for key, value in lotr_books.items():
    my_df = words_info(value)
    for c in replace_couples:
        my_df = replace_pseudonym(my_df, c[0], c[1])
    my_df = my_df.rename(columns={'special_counter':'cnt_'+str(key),'full_counter':'lower_cnt_'+str(key)})
    df_list.append(my_df)

df_final = ft.reduce(lambda left, right: pd.merge(left, right, on=['word', 'special_view'], how='outer'), df_list)
df_final = df_final.fillna(0)

#     tom in HOBBIT =/= tom in LOTR
#     bill in HOBBIT =/= bill in LOTR
df_final.loc[df_final['word'] == 'tom', ['cnt_0']] = 0
df_final.loc[df_final['word'] == 'bill', ['cnt_0']] = 0
df_final.loc[df_final['word'] == 'bill', ['cnt_7']] = 0
df_final.loc[df_final['word'] == 'merry', ['cnt_7']] = 0
df_final.loc[df_final['word'] == 'merry', ['cnt_0']] = 0

first_no = 0
books_cnt = len(lotr_books)

df_final['full_cnt'] = sum([df_final['lower_cnt_'+str(k)] for k in range(first_no,books_cnt)])
df_final['capital_full_cnt'] = sum([df_final['cnt_'+str(k)] for k in range(first_no,books_cnt)])

df_final


,word,lower_cnt_0,cnt_0,special_view,lower_cnt_1,cnt_1,lower_cnt_2,cnt_2,lower_cnt_3,cnt_3,...,lower_cnt_5,cnt_5,lower_cnt_6,cnt_6,lower_cnt_7,cnt_7,lower_cnt_8,cnt_8,full_cnt,capital_full_cnt
0,1,5.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,16.0,0.0,24.0,0.0
1,10,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,12.0,0.0
2,45,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0
3,9,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0
4,a,1959.0,0.0,0,1919.0,0.0,1652.0,0.0,1663.0,0.0,...,1167.0,0.0,1298.0,0.0,267.0,0.0,692.0,0.0,11964.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16787,éored11,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
16788,éoredwith,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
16789,éorod,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
16790,éothéod,0.0,0.0,Éothéod,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,44.0,44.0,44.0,44.0


In [12]:
df_capital = df_final[['word','special_view','capital_full_cnt']+['cnt_'+str(k) for k in range(first_no,books_cnt)]][df_final['full_cnt']*0.70 <= df_final['capital_full_cnt']].sort_values(['capital_full_cnt'], ascending = False).reset_index()
df_capital2 = df_capital.drop(['index'], axis = 1).rename(columns={'special_view':'name'})

# list of geographic objects and other non-character names with first capital letter
not_names = [
    'entwives', 'ents', 'ent', 'rohirrim', 'orcs', 'elves', 'riders', 'nazgûl', 'halflings',
    
    'isen', 'shire', 'gondor', 'rohan', 'mordor', 'minas', 'bree', 'mirkwood', 'morgul', 'rivendell', 'tirith', 'moria',
    'orthanc', 'anduin', 'fangorn', 'hobbiton', 'edoras', 'lothlórien', 'ithilien', 'bag', 'city', 'dale',
    'emyn', 'muil', 'osgiliath', 'bywater', 'brandywine', 'buckland', 'weathertop', 'barad', 'fords', 'dol', 'guldur',
    'amon', 'westfold', 'hornburg', 'misty', 'ford', 'barrow',
    
    'mr', 'baggins', 'bombadil', 'lady', 'council', 'prince', 'gamgee', 'sting', 'steward', 'oakenshield', 'ring',
    'durin', 'isildur', 'elendil',
    'oh', 'nay', 'ah', 'alas', 'author', 'appendix', 'deeping'
]
df_capital2 = df_capital2[~df_capital2['word'].str.contains("|".join(not_names)) ].reset_index().drop(['index'], axis = 1)

df_capital_top = df_capital2[df_capital2['capital_full_cnt'] >= 30].rename(columns={'capital_full_cnt':'full_cnt'})



In [7]:
# to add characters's race

def race_by_name(races, name):
    for key, name_list in races.items():
        if name in name_list:
            return key

races = {}

races['hobbit'] = ['frodo', 'sam', 'bilbo', 'pippin', 'gollum', 'merry', 'gaffer', 'maggot', 'cotton', 'nob']
races['elf'] = ['legolas', 'elrond', 'galadriel', 'haldir', 'celeborn', 'glorfindel', 'gildor']
races['maya'] = ['gandalf', 'saruman', 'tom', 'sauron', 'goldberry']
races['man'] = ['aragorn', 'faramir', 'boromir', 'éomer', 'théoden', 'denethor', 'éowyn', 'butterbur', 
                'beorn', 'wormtongue', 'bard', 'beregond', 'cirion', 'eorl', 'éothéod', 'elfhelm', 'grimbold', 'théodred']
races['dwarf'] = ['gimli', 'thorin', 'balin', 'bombur', 'fili', 'glóin', 'kili', 'dori']
races['orc'] = ['uglúk', 'shagrat', 'grishnákh']
races['ent'] = ['treebeard']
races['animal'] = ['smaug', 'shadowfax', 'bill', 'shelob']



In [8]:
df_capital_top['race'] = df_capital_top['word'].apply(lambda x: race_by_name(races, x) )
df_capital_top

,word,name,full_cnt,cnt_0,cnt_1,cnt_2,cnt_3,cnt_4,cnt_5,cnt_6,cnt_7,cnt_8,race
0,frodo,Frodo,1988.0,0.0,622.0,463.0,24.0,474.0,16.0,375.0,2.0,12.0,hobbit
1,gandalf,Gandalf,1475.0,187.0,179.0,284.0,334.0,32.0,246.0,90.0,1.0,122.0,maya
2,sam,Sam,1295.0,0.0,211.0,197.0,13.0,407.0,3.0,458.0,3.0,3.0,hobbit
3,aragorn,Aragorn,1000.0,0.0,181.0,231.0,291.0,16.0,185.0,64.0,0.0,32.0,man
4,bilbo,Bilbo,909.0,554.0,186.0,95.0,2.0,6.0,1.0,32.0,9.0,24.0,hobbit
5,pippin,Pippin,737.0,0.0,156.0,67.0,199.0,0.0,248.0,59.0,0.0,8.0,hobbit
6,gollum,Gollum,670.0,82.0,41.0,26.0,1.0,454.0,2.0,23.0,0.0,41.0,hobbit
7,merry,Merry,571.0,0.0,152.0,42.0,134.0,0.0,149.0,94.0,0.0,0.0,hobbit
8,gimli,Gimli,398.0,0.0,0.0,120.0,192.0,1.0,59.0,18.0,0.0,8.0,dwarf
9,saruman,Saruman,393.0,0.0,3.0,47.0,185.0,2.0,7.0,35.0,0.0,114.0,maya


In [10]:
# to pivot counter-columns to rows

df_capital_top_melt = df_capital_top.drop(['word'], axis = 1)
df_capital_top_melt = df_capital_top_melt.melt(id_vars=["name", "race"]).rename(columns={'name':'character','variable':'book_no','value':'cnt'})
df_capital_top_melt = df_capital_top_melt[df_capital_top_melt['book_no'] != 'full_cnt']
df_capital_top_melt['book_no'] = df_capital_top_melt['book_no'].apply(lambda x: int(x[-1]))
df_capital_top_melt = df_capital_top_melt.merge(df_lotr[['number','book_name', 'volume_name']], left_on='book_no', right_on='number')
df_capital_top_melt = df_capital_top_melt.reset_index().drop(['index'], axis=1)
df_capital_top_melt


,character,race,book_no,cnt,number,book_name,volume_name
0,Frodo,hobbit,0,0.0,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
1,Gandalf,maya,0,187.0,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
2,Sam,hobbit,0,0.0,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
3,Aragorn,man,0,0.0,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
4,Bilbo,hobbit,0,554.0,0,"The Hobbit, or There and Back Again","The Hobbit, or There and Back Again"
...,...,...,...,...,...,...,...
499,Dori,dwarf,8,0.0,8,The Third Age,"Unfinished Tales of Numenor and Middle-Earth, ..."
500,Shelob,animal,8,0.0,8,The Third Age,"Unfinished Tales of Numenor and Middle-Earth, ..."
501,Grishnákh,orc,8,0.0,8,The Third Age,"Unfinished Tales of Numenor and Middle-Earth, ..."
502,Gildor,elf,8,0.0,8,The Third Age,"Unfinished Tales of Numenor and Middle-Earth, ..."


In [230]:
df_capital_top_melt.to_csv('lotr_characters_top_melt.csv')